In [35]:
%matplotlib inline
import shutil, os, sys, glob
import numpy as np
from matplotlib import pyplot as plt
import ipyvolume as ipv
#
sys.path.append('/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/src/')
import cryoemio
import imutils
import dataviz
import mrcutils

# some useful paths and parameters

In [50]:
input_dir = '/gpfs/slac/cryo/fs1/g/ML/cryoEM-notebooks/data/20181005-rib-TEM4/LocalRes/bin2_of_bin6mb_A_fscw'
body_dir  = '/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw'
# files
keyword='bodies_bin2_of_bin6_mbA_'
input_mrc    = input_dir+'/relion_locres_filtered.mrc'
output_mask  = body_dir+'/'+keyword
#

# load map and retrieve standard deviation

In [51]:
data = mrcutils.mrc2data(input_mrc)
input_std = np.std(data)
print(input_std)

0.03112948


# visualize effect of thresholding

In [52]:
data_thresh = mrcutils.mrc_select(input_mrc, mode='above_value', value=1*input_std)

In [53]:
ipv.quickvolshow(data_thresh, level=[1*input_std, 3*input_std], opacity=0.03, level_width=1*input_std, data_min=np.min(data_thresh), data_max=np.max(data_thresh))

# visualize effect of binarizing

In [54]:
mask = mrcutils.data2mask(data_thresh, sigma_blur=0, threshold=0.1)
ipv.quickvolshow(data_thresh, level=[0, 1], opacity=0.03)

# define bodies: segment differential thresholding map

The idea is to
- blur mask_dry a lot
- blur mask_fat a little
- define the main body as the intersection of both
- define the other bodies as what remains of mask_fat after removing the intersection
- then segment them

In other words:

$B_0 = M_{fat} \cap M_{dry}$

$\bigcup_{i>1}^{K} B_{i} = M_{fat} \setminus B_0 $

In [55]:
data_dry = mrcutils.mrc_select(input_mrc, mode='above_value', value=5*input_std)
mask_dry = mrcutils.data2mask(data_dry, sigma_blur=15, threshold=0.01)
data_fat = mrcutils.mrc_select(input_mrc, mode='above_value', value=1*input_std)
mask_fat = mrcutils.data2mask(data_fat, sigma_blur=1, threshold=0.1)

In [56]:
body0 = np.minimum(mask_dry,mask_fat)
bodyK = mask_fat - body0

In [57]:
ipv.quickvolshow(body0, level=[0, 1], opacity=0.03)

In [58]:
ipv.quickvolshow(bodyK, level=[0, 1], opacity=0.03)

In [59]:
mrcutils.data2mrc(output_mask+'body_0.mrc',body0,mrc_template=input_mrc)
mrcutils.data2mrc(output_mask+'body_K.mrc',bodyK,mrc_template=input_mrc)

In [60]:
import skimage.morphology as morphology
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
fp=60
distance = ndimage.distance_transform_edt(bodyK)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((fp,fp,fp)), labels=bodyK)
markers = morphology.label(local_maxi)
labels_ws = watershed(-distance, markers, mask=bodyK)

In [61]:
ipv.quickvolshow(labels_ws, level=[0, float(np.max(labels_ws))], opacity=0.03)

In [62]:
mrcutils.data2mrc(output_mask+'body_K_watershed_seg.mrc',labels_ws,mrc_template=input_mrc)
# break down: one mask per segment
mrcutils.seg2mask(output_mask+'body_K_watershed_seg.mrc', output_mask+'body_K_watershed', sigma_blur = mask_blur,sort='volume',verbose=True)

/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_K_watershed1.mrc > volume = 890451.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_K_watershed2.mrc > volume = 839398.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_K_watershed3.mrc > volume = 823368.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_K_watershed4.mrc > volume = 742336.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_K_watershed5.mrc > volume = 323400.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_K_watershed6.mrc > volume = 152854.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-r

At that point, a visual inspection (in Chimera) leads to following decision:
- body 0: 0,8,10,12,13,14,15,16,17
- body 1: 1,7
- body 2: 2
- body 3: 3,4,5,6,9
- body 4: 11

In [63]:
# final body 0 
shutil.copyfile(output_mask+'body_0.mrc', 'tmp.mrc')
for i in [8,10,12,13,14,15,16,17]:
    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
    os.remove('tmp.mrc')
    os.rename('tmp2.mrc', 'tmp.mrc')
mrcutils.mrc2mask('tmp.mrc', output_mask+'body_0_final.mrc', sigma_blur=0., threshold=0.1)
os.remove('tmp.mrc')
# final body 1
shutil.copyfile(output_mask+'body_K_watershed1.mrc', 'tmp.mrc')
for i in [7]:
    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
    os.remove('tmp.mrc')
    os.rename('tmp2.mrc', 'tmp.mrc')
mrcutils.mrc2mask('tmp.mrc', output_mask+'body_1_final.mrc', sigma_blur=0., threshold=0.1)
os.remove('tmp.mrc')
# final body 2
shutil.copyfile(output_mask+'body_K_watershed2.mrc', output_mask+'body_2_final.mrc')
# final body 3
shutil.copyfile(output_mask+'body_K_watershed3.mrc', 'tmp.mrc')
for i in [4,5,6,9]:
    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
    os.remove('tmp.mrc')
    os.rename('tmp2.mrc', 'tmp.mrc')
mrcutils.mrc2mask('tmp.mrc', output_mask+'body_3_final.mrc', sigma_blur=0., threshold=0.1)
os.remove('tmp.mrc')
# final body 4
shutil.copyfile(output_mask+'body_K_watershed11.mrc', output_mask+'body_4_final.mrc')

'/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin2_of_bin6mb_A_fscw/bodies_bin2_of_bin6_mbA_body_4_final.mrc'